In [1]:
import pandas as pd
import requests
import time

In [2]:
df = pd.read_csv("latitude_longitude.csv")
df

,iata_code,latitude_deg,longitude_deg,elevation_m
0,JFK,40.639447,-73.779317,2.765000
1,LAX,33.942501,-118.407997,32.489697
2,DFW,32.896801,-97.038002,175.985489
3,OKC,35.393101,-97.600700,393.050842
4,OGG,20.898543,-156.431212,7.000000
...,...,...,...,...
360,FNL,40.451816,-105.011326,NaN
361,BKG,36.532083,-93.200544,NaN
362,AKN,58.676493,-156.648691,NaN
363,HYA,41.669337,-70.280359,NaN


In [3]:
df['elevation_m'] = pd.Series(dtype='float')
backlog = []

for idx in range(0, len(df), 100):
    locations = []

    for j in range(idx, min(len(df), idx + 100)):
        location = str(df.iloc[j]['latitude_deg']) + "," + str(df.iloc[j]['longitude_deg'])
        locations.append(location)

    # send request for every 100 locations
    r = requests.post("https://api.opentopodata.org/v1/ned10m,aster30m", json={
        "locations": '|'.join(locations)
    })
    data = r.json()

    if data["status"] != "OK":
        break

    j = idx
    for location in data["results"]:
        df.at[j, 'elevation_m'] = location["elevation"]
        j += 1

    # api accepts only 1 call every 1 second
    time.sleep(1)

df

,iata_code,latitude_deg,longitude_deg,elevation_m
0,JFK,40.639447,-73.779317,2.765000
1,LAX,33.942501,-118.407997,32.489697
2,DFW,32.896801,-97.038002,175.985489
3,OKC,35.393101,-97.600700,393.050842
4,OGG,20.898543,-156.431212,7.000000
...,...,...,...,...
360,FNL,40.451816,-105.011326,1522.290894
361,BKG,36.532083,-93.200544,385.358429
362,AKN,58.676493,-156.648691,19.551983
363,HYA,41.669337,-70.280359,9.054512


In [4]:
df.to_csv("latitude_longitude.csv", index=False)